# Representativity - Ecologically important area coverage 1.3

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [10]:
# Union the dissolved MPAs with the dissolved ecologically imp areas

in_feature_1 = "WDPA_MPA_dis"
in_feature_2 = "KBA_EBSA_mol_Carib_dis"
in_feature_3 = "Caribbean_detailed_subregions_mol"
out_feature = "\\MPA_dis_KBA_EBSA_union_Rep1_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Union(
    in_features=[in_feature_1,in_feature_2,in_feature_3],
    out_feature_class= output_gdb + out_feature,
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\MPA_dis_KBA_EBSA_union_Rep1_3'>

In [11]:
# Repair geoemtry on union, calculate area, and export to excel

# Set a folder to store the output tables
table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"

in_feature = "MPA_dis_KBA_EBSA_union_Rep1_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def repair_geo_area_export():
    #repair geometry
    arcpy.management.RepairGeometry(
        in_features= in_feature,
        delete_null="DELETE_NULL",
        validation_method="OGC"
    )
            
    # insert "area" field (double) and calculate area in km2 in Mollweide
    arcpy.management.CalculateGeometryAttributes(
        in_features= in_feature,
        geometry_property="area AREA",
        length_unit="",
        area_unit="SQUARE_KILOMETERS",
        coordinate_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
        coordinate_format="SAME_AS_INPUT"
    )
    
    # Export tables to excel
    # Not sure why I had to use the following two lines instead of just using the TableToExcel expression, but I did
    excel_filename = in_feature + ".xlsx"
    excel_filepath = os.path.join(table_outputs_folder, excel_filename)  # Build the full file path

    arcpy.conversion.TableToExcel(
        Input_Table= in_feature,
        Output_Excel_File= excel_filepath,
        Use_field_alias_as_column_header="NAME",
        Use_domain_and_subtype_description="CODE"
    )
    
repair_geo_area_export()


In [12]:
# Convert the output features to a Pandas DataFrame
df_MPA_union = pd.DataFrame.spatial.from_featureclass("MPA_dis_KBA_EBSA_union_Rep1_3")


In [13]:
# Display the DataFrame
df_MPA_union.head()

,OBJECTID,FID_WDPA_MPA_dis,FID_Caribbean_detailed_subregions_mol,ORIG_FID,Area,Subregion,FID_KBA_EBSA_mol_Carib_dis,SHAPE
0,1,-1,1,2,1.249407e+06,Greater Antilles,-1,"{""hasZ"": true, ""rings"": [[[-7955414.684900001,..."
1,2,-1,2,2,1.364962e+06,Gulf of Mexico,-1,"{""hasZ"": true, ""rings"": [[[-8035203.846999999,..."
2,3,-1,4,2,1.987861e+05,Florida,-1,"{""hasZ"": true, ""rings"": [[[-7044181.704399999,..."
3,4,-1,6,2,6.619637e+05,Bahamian,-1,"{""hasZ"": true, ""rings"": [[[-6945139.6688, 3626..."
4,5,-1,8,2,3.538951e+05,Eastern Caribbean,-1,"{""hasZ"": true, ""rings"": [[[-5660916.9866, 2276..."


In [27]:
# Make pivot tables
pivot_subregion = df_MPA_union.pivot_table(
     index = "Subregion",
     values = "Area",
     aggfunc = "sum"
).reset_index()
pivot_subregion.columns = ["Subregion", "Total_area"]
#print(pivot_subregion)

pivot_KBA_EBSA = df_MPA_union[df_MPA_union["FID_KBA_EBSA_mol_Carib_dis"] != -1].pivot_table(
    index = "Subregion",
    values = "Area",
    aggfunc = "sum"
).reset_index()
pivot_KBA_EBSA.columns = ["Subregion", "KBA_EBSA_area"]
#print(pivot_KBA_EBSA)

pivot_MPA = df_MPA_union[df_MPA_union["FID_WDPA_MPA_dis"] != -1].pivot_table(
    index = "Subregion",
    values = "Area",
    aggfunc = "sum"
).reset_index()
pivot_MPA.columns = ["Subregion", "MPA_area"]
#print(pivot_MPA)

pivot_MPA_KBA_EBSA_int = df_MPA_union[(df_MPA_union["FID_WDPA_MPA_dis"] != -1) & (df_MPA_union["FID_KBA_EBSA_mol_Carib_dis"] != -1)].pivot_table(
    index = "Subregion",
    values = "Area",
    aggfunc = "sum"
).reset_index()
pivot_MPA_KBA_EBSA_int.columns = ["Subregion", "MPA_KBA_EBSA_int_area"]
#print(pivot_MPA_KBA_EBSA_int)

# Merge the pivot tables into a single pivot table. The merge should be based on Subregion
KBA_EBSA_coverage = pd.merge(pivot_subregion, pivot_MPA, on="Subregion", how="left", validate = "1:1")
KBA_EBSA_coverage = pd.merge(KBA_EBSA_coverage, pivot_KBA_EBSA, on="Subregion", how="left", validate = "1:1")
KBA_EBSA_coverage = pd.merge(KBA_EBSA_coverage, pivot_MPA_KBA_EBSA_int, on="Subregion", how="left", validate = "1:1")
#print(KBA_EBSA_coverage)

#add column for the percent KBA_EBSA coverage per subregion
KBA_EBSA_coverage.insert(5, "%KBA_EBSA_coverage", KBA_EBSA_coverage["MPA_KBA_EBSA_int_area"] / KBA_EBSA_coverage["KBA_EBSA_area"] * 100)
print(KBA_EBSA_coverage)


                Subregion    Total_area       MPA_area  KBA_EBSA_area  \
0                Bahamian  7.109443e+05   47434.279020    2801.288318   
1       Eastern Caribbean  8.277024e+05  156780.054923  394656.631215   
2                 Florida  2.152991e+05   16395.806871    3659.854140   
3        Greater Antilles  1.391880e+06   69691.401433  107967.222547   
4                 Guianan  4.001308e+05    3371.326517  397031.503126   
5          Gulf of Mexico  1.404182e+06   20149.642202   28044.922869   
6      Southern Caribbean  5.436105e+05   34024.578178   43975.769529   
7  Southwestern Caribbean  7.502861e+05   89024.215528  111028.240617   
8       Western Caribbean  3.446225e+05   80278.382613  254415.070271   

   MPA_KBA_EBSA_int_area  %KBA_EBSA_coverage  
0            1255.032457           44.801974  
1           77629.334152           19.670095  
2            3542.745525           96.800184  
3           35185.621275           32.589170  
4            3127.847292          

In [28]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Representativity"
table = KBA_EBSA_coverage
output_name = "MPA_dis_KBA_EBSA_union_Rep1_3_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))